In [27]:
import pandas as pd
from sqlalchemy import create_engine

# Put postgreSQL credeitials into sql_login.py
from sql_login import USERNAME as USERNAME
from sql_login import PASSWORD as PASSWORD

In [28]:
# This function outputs a data frame from a csv file from https://apps.fas.usda.gov/
# It expects data from four quarters from 2015 to 2020
def fas_csv_to_pd_df(file_name):
    input_df = pd.read_csv(file_name)
    out_df = pd.DataFrame(columns = ['Year','Quarter','Value','Qty'])
    year_list = []
    quarter_list=[]
    for j in range(2015,2020):
        for i in range(1,5):
            quarter_list.append(i)
            year_list.append(j)

    out_df['Year'] = year_list
    out_df['Quarter'] = quarter_list

    value_list = []
    quantity_list = []
    for i in range(0,5):
        for j in range(6,14,2):   
            value_list.append(input_df.iloc[i,j].replace(',',''))
            quantity_list.append(input_df.iloc[i,j+1].replace(',',''))
    out_df['Value'] = value_list
    out_df['Qty'] = quantity_list
    out_df.columns = map(str.lower, out_df.columns)
    return out_df
    


In [31]:
# Rearrange data for corn
corn_df = fas_csv_to_pd_df('Resources/Corn.csv')
corn_df
# Rearrange data for limes

# Rearrange data from mangos
mango_df = fas_csv_to_pd_df('Resources/Mango_cleaned.csv')
mango_df
# Rearrange data for sorghum

corn_df[['value','qty']] = corn_df[['value','qty']].apply(pd.to_numeric)
mango_df[['value','qty']] = mango_df[['value','qty']].apply(pd.to_numeric)

corn_df['price'] = corn_df['value']/corn_df['qty']
mango_df['price'] = mango_df['value']/mango_df['qty']

## Connect to postgreSQL and import the dataframe

In [32]:
connection_string = USERNAME+':'+PASSWORD+"@localhost:5432/avocados"
engine = create_engine(f'postgresql://{connection_string}')
conn = engine.engine.connect()

In [37]:
# Drop avocado price table if it exists
conn.execute('DROP TABLE IF EXISTS corn_fsda;')
conn.execute('DROP TABLE IF EXISTS mango_fsda;')

In [39]:
# Create tables to hold quarterly import data
conn.execute(
"""
CREATE TABLE corn_fsda (
    quarter INT,
    year INT,
    qty DECIMAL(20,2),
    price double precision,
    primary key (quarter,year)
);
"""
)
conn.execute(
"""
CREATE TABLE mango_fsda (
    quarter INT,
    year INT,
    qty DECIMAL(20,2),
    price double precision,
    primary key (quarter,year)
);
"""
)

In [40]:
corn_df[['year','quarter','price']].to_sql(name='corn_fsda', con=engine, if_exists='append', index=False)
mango_df[['year','quarter','price']].to_sql(name='mango_fsda', con=engine, if_exists='append', index=False)